In [2]:
import placekey as pk
from placekey.api import PlacekeyAPI
import h3 as h3
import pandas as pd
import json
import os

# Playing around with the placekey library and API

Starting with some initial testing.

## Generating a placekey for my apartment

In [3]:
coord = (38.8753221,-77.008219) # Location of my apartment at 1201 Half St. SE

apt_placekey = pk.geo_to_placekey(*coord)

print(f"Kevin's apartment placekey: {apt_placekey}")

Kevin's apartment placekey: @63r-6cv-w6k



<strong><em>Pretty cool! Great runtime.</em></strong>

## Let's start playing around with some real data

In [4]:
# For posterity and a possible performance improvement when reading csv
blight_column_dtype = { 'X': float, 'Y': float, 'violation_address': str }
property_column_dtype = { 'X': float, 'Y': float, 'address': str }

# Read both datasets into dataframes
blight_violations_df = pd.read_csv('./data/Blight_Violations.csv', low_memory=False, dtype=blight_column_dtype)
property_sales_df = pd.read_csv('./data/Property_Sales.csv', low_memory=False, dtype=property_column_dtype)

### We need to clean this up a bit
<strong><em>This dataset has some values we don't necessarily need for the scope of this writeup</em></strong>

In [5]:
print("---BLIGHT----\n", blight_violations_df.head(), "\n\n---PROPERTY---\n", property_sales_df.head())

---BLIGHT----
            X          Y  ticket_id ticket_number  \
0 -83.072568  42.383357      18645   05001700DAH   
1 -83.072474  42.383400      18646   05001701DAH   
2 -83.115057  42.359928      18648   05001703DAH   
3 -83.128037  42.393455      18649   05001704DAH   
4 -83.134466  42.389668      18650   05001705DAH   

                                      agency_name inspector_name  \
0  Buildings, Safety Engineering & Env Department   Orbie Gailes   
1  Buildings, Safety Engineering & Env Department   Orbie Gailes   
2  Buildings, Safety Engineering & Env Department   Orbie Gailes   
3  Buildings, Safety Engineering & Env Department   Orbie Gailes   
4  Buildings, Safety Engineering & Env Department   Orbie Gailes   

     violator_name  violation_street_number violation_street_name  \
0        Dean Byrd                      601                  KING   
1  Cynthia Roberts                      607                  KING   
2    Dannny Barnes                     4066             

### We really only want the (x, y) coordinates plus the stringified full address

In [6]:
# Drop all columns not defined in the dtype dictionary above
blight_violations_df = blight_violations_df.filter(blight_column_dtype.keys()).truncate(before=0, after=1000)
property_sales_df = property_sales_df.filter(property_column_dtype.keys()).truncate(before=0, after=1000)

# Let's take a look at the results
print(blight_violations_df.head())
print(property_sales_df.head())

           X          Y violation_address
0 -83.072568  42.383357          601 KING
1 -83.072474  42.383400          607 KING
2 -83.115057  42.359928     4066 COLUMBUS
3 -83.128037  42.393455     3005 PASADENA
4 -83.134466  42.389668     4024 CLEMENTS
           X          Y            address
0 -83.224446  42.388139  14006 GLASTONBURY
1 -83.210323  42.438587  19951 ASBURY PARK
2 -83.209319  42.415134  16881 ASBURY PARK
3 -83.161543  42.351916      10322 TIREMAN
4 -83.074014  42.346942       4158 LINCOLN



<strong><em>Sweet! Now we have some cleaner data to work with and really harness PlaceKey's API.</em></strong>

In [7]:
# Get API key from environment variables
placekey_api_key = os.getenv('PLACEKEY_API_KEY')

# Create connection to Placekey API
pk_api = PlacekeyAPI(placekey_api_key)

In [8]:
# We need to reset the index to get our query_id, then rename the columns to conform
# with Placekey's query params so let's write a function to handle this

def df_to_api_query(df: pd.DataFrame, column_map: dict):
    # Renaming our columns
    df = df.reset_index().rename(columns=column_map)

    # Setting default region and country code (we assume Michigan and US for this particular dataset)
    df['region'] = 'MI'
    df['city'] = 'Detroit'
    df['iso_country_code'] = 'US'

    # We need to convert query_id to string to conform with Placekey's API docs
    df['query_id'] = df['query_id'].astype(str)

    # Return jsonified dataframe
    return json.loads(df.to_json(orient='records'))


# Use our function to convert dataframes to json
blight_violations_json = df_to_api_query(df=blight_violations_df, \
                                         column_map={'index': 'query_id', 'violation_address': 'street_address', 'X': 'longitude', 'Y': 'latitude'})
property_sales_json = df_to_api_query(df=property_sales_df, \
                                      column_map={'index': 'query_id', 'address': 'street_address', 'X': 'longitude', 'Y': 'latitude' })

# Let's check our work
print(blight_violations_df.head())

# Finally, let's make this API call
blight_response = pk_api.lookup_placekeys(blight_violations_json, verbose=True)
property_response = pk_api.lookup_placekeys(property_sales_json, verbose=True)


print('BLIGHT: ', len(blight_response), 'PROPERTY: ', len(property_response))

           X          Y violation_address
0 -83.072568  42.383357          601 KING
1 -83.072474  42.383400          607 KING
2 -83.115057  42.359928     4066 COLUMBUS
3 -83.128037  42.393455     3005 PASADENA
4 -83.134466  42.389668     4024 CLEMENTS


2024-02-10 12:43:01,258	INFO	Processed 1000 items
2024-02-10 12:43:01,813	INFO	Processed 1001 items
2024-02-10 12:43:01,814	INFO	Done
2024-02-10 12:43:32,492	INFO	Processed 1000 items
2024-02-10 12:43:32,887	INFO	Processed 1001 items
2024-02-10 12:43:32,888	INFO	Done


BLIGHT:  1001 PROPERTY:  1001


<strong><em>Wowza!! That was pretty cool. Obviously, a more records, this would take quite a while.</em></strong>

<strong><em>In the long run, though, runtime will be much more efficient when joining these datasets</em></strong>

In [ ]:
# Now we can convert the API response to a dataframe
blight_pk_df = pd.read_json(json.dumps(blight_response), dtype={'query_id': str})
property_pk_df = pd.read_json(json.dumps(property_response), dtype={'query_id': str})

# Let's create another function to handle merging the placekey response with the original df
def merge_placekey_with_df(pk_df: pd.DataFrame, org_df: pd.DataFrame):
    org_df = org_df.reset_index()
    org_df['index'] = org_df['index'].astype(str)
    
    with_pk_df = pd.merge(org_df, pk_df, left_on="index", right_on="query_id", how='left')

    return with_pk_df

blight_pk_merged = merge_placekey_with_df(blight_pk_df, blight_violations_df)
property_pk_merged = merge_placekey_with_df(property_pk_df, property_sales_df)

In [35]:
# Simulate just a WHERE (within 1100m) key
blight_pk_merged['where_pk_7'] = blight_pk_merged['placekey'].str[-12:-2]
property_pk_merged['where_pk_7'] = property_pk_merged['placekey'].str[-12:-2]

# Aggregate number of violations per placekey
blight_pk_agg = blight_pk_merged.groupby('where_pk_7').agg(
    { 'query_id' : 'count' }).reset_index().rename({'query_id' : 'violations_within_1100_meters'}, axis=1)

# Finally, lets outer join these two data frames to really harness the power of placekey
joined_blight_property_df = property_pk_merged.merge(blight_pk_agg, how='inner', on='where_pk_7')

print(joined_blight_property_df)

# Runtime is great when joining on placekey!
joined_blight_property_df.to_csv('joined.csv', sep='\t')

    index          X          Y                   address query_id  \
0       2 -83.209319  42.415134         16881 ASBURY PARK        2   
1       4 -83.074014  42.346942              4158 LINCOLN        4   
2       5 -83.093178  42.390215            724 BURLINGAME        5   
3       8 -83.218099  42.416632             17167 FENMORE        8   
4      16 -83.215977  42.416556              17161 HARLOW       16   
..    ...        ...        ...                       ...      ...   
425   986 -83.010869  42.431333             18645 ALGONAC      986   
426   990 -83.006921  42.345242  6533 E JEFFERSON 94/305J      990   
427   993 -83.113946  42.354402             6733 HARTFORD      993   
428   998 -83.234435  42.361752            9224 PLAINVIEW      998   
429  1000 -83.159045  42.429704             18901 INDIANA     1000   

                   placekey  where_pk_7  violations_within_1100_meters  
0           23d@63v-458-gx5  @63v-458-g                              3  
1           2

### We can now see the number of violations within 1100 meters (due to the length of the shared prefix) per property in Detroit

## Conclusion
<strong><em>While calling Placekey's API with 1M+ records can be time consuming (understandable), the speed of joining datasets with a placekey seems to be worth it! Pretty cool experiment! When looking at the joined dataset, we can see that addresses with similar/same longitude and latitude (but different addresses) have identical placekeys! Really interesting!
</em></strong>